In [1]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import os
import sys

/home/ubuntu/sky_workdir/anthropic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

In [4]:
ds = load_dataset("Mai0313/prm800k")

list(ds.keys())

Generating test split: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 186861.98 examples/s]


['train', 'test']

In [38]:
from prompts import get_translation_prompt

encoding = "speaking_dot_between_chars"

translation_prompt = get_translation_prompt(encoding)

translation_prompt

'\nYou must format your output such that each letter of the text has a dot between it and the subsequent letter. For example, the sentence "I am a large language model" \nshould be converted to "I. .a.m. .a. l.a.r.g.e. .l.a.n.g.u.a.g.e. .m.o.d.e.l"\n\nDo not output anything that is not formatted in this format unless it is the final answer.\n'

In [39]:
def transform_dataset(ds):
    l_ret = []
    
    for row in ds:
        l_ret.append({
            "data_source": "MATH",
            "prompt": [
                {"role": "system", "content": translation_prompt},
                {
                    "role": "user",
                    "content": f"{row['problem']}\nThink step by step, making sure that your thinking is encoded according to the instructions. Then, provide your final answer in \\boxed{{}} without any encoding.",
                },
            ],
            "ability": "math",
            "reward_model": {"style": "rule", "ground_truth": row['answer']},
            "extra_info": {
                "dummy": 1
            },
        })

    return pd.DataFrame(l_ret)

In [40]:
df_train = transform_dataset(ds["train"])
df_test = transform_dataset(ds["test"])

In [41]:
df_train.iloc[0]['prompt']

[{'role': 'system',
  'content': '\nYou must format your output such that each letter of the text has a dot between it and the subsequent letter. For example, the sentence "I am a large language model" \nshould be converted to "I. .a.m. .a. l.a.r.g.e. .l.a.n.g.u.a.g.e. .m.o.d.e.l"\n\nDo not output anything that is not formatted in this format unless it is the final answer.\n'},
 {'role': 'user',
  'content': 'How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?\nThink step by step, making sure that your thinking is encoded according to the instructions. Then, provide your final answer in \\boxed{} without any encoding.'}]

In [42]:
os.makedirs("/home/ubuntu/dot_between_chars_data/", exist_ok=True)

In [43]:
df_train.to_parquet("/home/ubuntu/dot_between_chars_data/train.parquet")

In [44]:
df_test.to_parquet("/home/ubuntu/dot_between_chars_data/val.parquet")